In [1]:
from moviepy.editor import VideoFileClip
import os
import threading
from queue import Queue
from tqdm import tqdm

def get_videos_list(directory=''):
    return [x for x in os.listdir(directory) if '.mp4' in x]

def reduce_fps(video_path, new_fps, output_directory=''):
    try:
        # Extract file name
        file_name = os.path.basename(video_path)
        file_name = file_name[file_name[:file_name.rfind('.')].rfind('_')+1:file_name.rfind('.')] + '.mp4'
    
        # Load the video
        video = VideoFileClip(video_path)
        
        # print the original FPS
        # print(f"Orignal FPS: {video.fps}")
        video = video.resize(height=360)
        
        # Write the modified video
        output_path = os.path.join(output_directory, file_name)
        video.write_videofile(
            output_path, 
            fps=new_fps,
            verbose=False, 
            logger=None,
            threads=6,
            ffmpeg_params=["-preset", "ultrafast", "-crf", "23"]
        )
        video.close()
    except Exception as e:
        print(f"Ошибка в {video_path}: {e}")
    
def reduce_fps_in_videos(videos_list, new_directory=True):
    if new_directory:
        if not os.path.exists('formatted_videos'):
            os.makedirs('formatted_videos')
        for video_ in videos_list:
            reduce_fps('downloaded_tiktoks/'+video_, new_fps=5, output_directory='formatted_videos')
    else:
        for video_ in videos_list:
            reduce_fps(video_, new_fps=5)

def worker(q, new_fps, output_directory):
    """Функция для потока: берет видео из очереди и обрабатывает."""
    while not q.empty():
        video_path = q.get()
        reduce_fps(video_path, new_fps, output_directory)
        q.task_done()
        
def process_videos_threaded(video_paths, new_fps=5, output_directory='', num_threads=4):
    """Запускает обработку видео в многопоточном режиме."""
    q = Queue()
    for path in video_paths:
        q.put('downloaded_tiktoks/'+path)

    # Создаем и запускаем потоки
    threads = []
    for _ in range(num_threads):
        t = threading.Thread(
            target=worker,
            args=(q, new_fps, output_directory),
            daemon=True
        )
        t.start()
        threads.append(t)

    # Ожидаем завершения (можно добавить прогресс-бар)
    q.join()
    print("Все видео обработаны!")
    
import time

t1 = time.perf_counter()
    
if __name__ == "__main__":
    video_paths = get_videos_list('downloaded_tiktoks')
    
    if not os.path.exists('formatted_videos'):
        os.makedirs('formatted_videos')
        
    already_formatted = get_videos_list('formatted_videos')
    video_paths = list(set(video_paths) - set(already_formatted))
    
    process_videos_threaded(video_paths, new_fps=5, num_threads=8, output_directory='formatted_videos')
    
t2 = time.perf_counter()

print(t2-t1)

Ошибка в downloaded_tiktoks/7504991241235336479.mp4: index -90723 is out of bounds for axis 0 with size 9278
Ошибка в downloaded_tiktoks/7500946401224920362.mp4: MoviePy error: failed to read the duration of file downloaded_tiktoks/7500946401224920362.mp4.
Here are the file infos returned by ffmpeg:

ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvi

Все видео обработаны!
10462.7726721


В среднем 1,44 секунды занимает обработка 1-го видео (1641 / 1137)